In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# reset defalult plotting values
plt.rcParams['figure.figsize'] = (15, 5)
plt.rc('font', family='sans-serif')
plt.rc('axes', labelsize=14)
plt.rc('axes', labelweight='bold')
plt.rc('axes', titlesize=16)
plt.rc('axes', titleweight='bold')
plt.rc('axes', linewidth=2)
plt.rc('xtick',labelsize=14)
plt.rc('ytick',labelsize=14)

# Lomb-Scargle Periodogram
## Finding the period of a periodic signal

<img src="media/lomb-scargle.png" width=900>

### Prof. Robert Quimby
&copy; 2020 Robert Quimby

## In this tutorial you will...

- Review Fourier decomposition of arbitrary periodic signals into weighted sums of sine and cosine terms
- Create a periodic signal and consider the power in each frequency
- Given a periodic signal, determine the power in each frequency
- Use a Lomb-Scargle periodogram to determine the period of a real astrophysical signal

For a more complete introduction, see [Jake VanderPlas' paper](https://ui.adsabs.harvard.edu/abs/2018ApJS..236...16V/abstract) on "Understanding the Lomb-Scargle Periodogram"

## Fourier Decomposition

Any periodic function, $f(x)$, can be expressed as a sum of sine and cosine waves of various amplitudes and frequencies as follows:

$$f(x) = {1 \over 2}a_0 + \sum_{n=1}^{\infty}a_n \cos(nx) + \sum_{n=1}^{\infty}b_n \sin(nx)  $$

(see http://mathworld.wolfram.com/FourierSeries.html)

## Fourier decomposition example: triangle waves

$$f(t) = {8 \over \pi^2} \sum_{n=1, 3, 5, ...}^{\infty}{(-1)^{(n-1)/2} \over n^2} \sin\left(2\pi n t \over P\right)  $$



In [ ]:
def triangle_wave(t, nterms, period=1):
    components = []
    for n in range(1, 2*nterms, 2):
        A = ????
        f = ????
        wave = A * np.sin(2 * np.pi * f * t)
        components.append(8 / np.pi**2 * wave)
    return np.array(components)

In [ ]:
t = np.linspace(0, 3, 1000)
components = triangle_wave(t, ????)
for component in components:
    plt.plot(t, component)

plt.grid()

## Power in each frequency

In [ ]:
# gather the frequencies and amplitudes for each individual
# sine wave contributing to the triangular waveform
b_n = []
f_n = []
nterms = 6
period = 1.0
for n in range(1, 2*nterms, 1):
    if n % 2 == 0:
        A = 0
    else:
        A = (-1)**( (n - 1) / 2 ) / n**2 
    b_n.append(A)
    f_n.append(n / period)
b_n = np.array(b_n)
f_n = np.array(f_n)

In [ ]:
# plot the coefficients
plt.bar( ???? )
plt.xlabel('Frequency')
plt.ylabel(r'$b_n$')
plt.grid()

In [ ]:
# plot the power in each coefficient
plt.bar( ???? )
plt.xlabel('Frequency')
plt.ylabel('Power/Frequency')
plt.grid()

## Given a discretely sampled signal, how can we determine the power in each frequency?

In [ ]:
# given only a `signal` sampled at times `t`
t = np.linspace(0, 10, 200)
signal = triangle_wave(t, nterms).sum(axis=0)
plt.scatter(t, signal)
plt.grid();

## Fit sinusoids of different frequencies to the data 

At times, $t$, create sinusoids of the form:
$$y = A \cos(2 \pi f t + \phi)$$

where $f$ is the test frequency and the amplitude, $A$, and phase shift, $\phi$ are model parameters to optimize against the data.

## Re-Cast in a form that is linear with the parameters $A$ and $\phi$

We can use the trig identity:
$$A\cos(\theta + \phi ) = B\cos \theta + C\sin \theta$$

where
$$\theta = 2 \pi f t \ ,$$

$$A = {B \over |B|}\sqrt{B^2 + C^2} \ ,$$

and
$$\phi = \arctan \left(-{C \over B}\right)$$

## Least-Squares fit to find best $A$ and $\phi$ at each frequency

With signal measurements, $y = [y_1, y_2, ..., y_N]$ at times, $t = [t_1, t_2, ..., t_N]$, and assuming a model of the form $y = B \cos{\theta} + C \sin{\theta}$,

$$
\left[ \begin{array}{c}
y_1  \\
y_2  \\
 \vdots  \\
y_N  \end{array} \right] = \left[ \begin{array}{ccc}
\cos{2 \pi f t_1} & \sin{2 \pi f t_1} \\
\cos{2 \pi f t_2} & \sin{2 \pi f t_2} \\
 \vdots & \vdots \\
\cos{2 \pi f t_N} & \sin{2 \pi f t_N}\end{array} \right] 
\left[ \begin{array}{c}
B \\
C \end{array} \right] $$


In [ ]:
# get the best fit amplitude and phase at each test frequency
freqs = ????
Y = np.matrix( ???? )
A = []
phi = []
for freq in freqs:
    thetas = 2 * np.pi * freq * t
    X = np.matrix( ???? )
    p = (X.T * X).I * (X.T * Y)
    B, C = p.A1
    A.append(B / np.abs(B) * np.hypot(B, C))
    phi.append(np.arctan(-C / B))
A = np.array(A)
phi = np.array(phi)

In [ ]:
plt.bar(f_n, b_n**2, label=r'$b_n^2$')
#plt.bar(freqs, A**2 / (A**2).max(), color='none', edgecolor='red', label=r'best-fit $A^2$', lw=3, hatch='//')
plt.yscale('log')
plt.xlabel('Frequency')
plt.ylabel(r'$A^2$')
plt.grid()
plt.legend();

## Use $\chi^2$ values to evaluate the relative power in each frequency

In [ ]:
def get_model(params, freq, times):
    A, phi = params
    return A * np.cos(2 * np.pi * freq * times + phi)

def get_chisq(params, times, freq, values):
    model = get_model(params, freq, times)
    return np.sum( (values - model)**2 )

In [ ]:
# calculate the chi-square value at each frequency fit
chisq = []
for i in range(freqs.size):
    chisq.append(get_chisq( ???? ))
chisq = np.array(chisq)

In [ ]:
# compare chi-square values relative to DC signal
chisq0 = ????
P = ????

In [ ]:
plt.bar(f_n, b_n**2, label=r'$b_n^2$')
plt.bar(freqs, A**2 / (A**2).max(), color='none', edgecolor='red'
        , label=r'best-fit $A^2$', lw=3, hatch='//')
#plt.bar(freqs, P, label=r'relative $\chi^2$', color='none', edgecolor='k', lw=3, hatch='\\\\')
plt.yscale('log')
plt.xlabel('Frequency')
plt.ylabel('Power/Frequency')
plt.grid()
plt.legend();

## Application to real data

In [ ]:
lc = np.genfromtxt('media/varstar.dat', names='mjd, mag, emag, flag')
t = lc['mjd']
signal = lc['mag']
plt.scatter(t, signal)
plt.xlabel('Time (MJD)')
plt.ylabel('Magnitude')
plt.grid();

In [ ]:
# get the best fit amplitude and phase at each test frequency
freqs = ????
Y = np.matrix( [[value] for value in signal] )
A2, chisq = [], []
for freq in freqs:
    thetas = 2 * np.pi * freq * t
    X = np.matrix( [[np.cos(theta), np.sin(theta)] for theta in thetas] )
    p = (X.T * X).I * (X.T * Y)
    B, C = p.A1
    A = B / np.abs(B) * np.hypot(B, C)
    A2.append(A**2)
    phi = np.arctan(-C / B)
    chisq.append(get_chisq( (A, phi), t, freq, signal))
chisq = np.array(chisq)
A2 = np.array(A2)
chisq0 = np.sum( (signal - np.mean(signal))**2 )
P = (chisq0 - chisq) / chisq0

In [ ]:
# plot the power in each frequency
plt.plot(freqs, P)
plt.xlabel('Frequency')
plt.ylabel('Power/Frequency')
plt.title('Lomb-Scargle Periodogram')
plt.grid();

## Lomb-Scargle in `astropy`

In [ ]:
import astropy.units as u
from astropy.timeseries import LombScargle

In [ ]:
t = lc['mjd'] * u.day
mags = lc['mag'] * u.mag
ls = LombScargle( ???? )
power = ????
plt.plot(freqs, power)
plt.xlabel('Frequency (1/d)')
plt.ylabel('Power/Frequency')
plt.grid();

In [ ]:
# include uncertainty
ls = LombScargle(t, mags, lc['emag'] * u.mag)

# automatically pick test frequencies
frequency, power = ????
plt.plot(frequency, power)
plt.xlim(0, 15)
plt.xlabel('Frequency ({})'.format(frequency[0].unit))
plt.ylabel('Power/Frequency')
plt.grid();

## Interpreting the periodogram to determine the period

- start with the 1 over the frequency with the maximum power
- but remember this is the sinusoid that best fits the data
  - your signal may not be a sinusoid!

In [ ]:
period = ????
phase = t % period
plt.scatter(phase, mags)
plt.gca().invert_yaxis()
plt.grid()
plt.xlabel('Phase ({})'.format(phase.unit))
plt.ylabel('Magnitude')